# 4 - PSD

Loads depth-frequency-time PSD data and then plots for various circumstances.

## Imports
Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import chi2
for i in range(2):
    %matplotlib notebook

In [129]:
# import data

adcp = 'Axis55'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018)
year = 2018
ds_in = xr.open_dataset(f'../Data/data/spectra/spectra_{adcp}_{year}_0.nc')

n_seg = ds_in.n_seg
if n_seg > 1:
    ds = [ds_in]
    for i in range(n_seg):
        if i > 0:
            ds_temp = xr.open_dataset(f'../Data/data/spectra/spectra_{adcp}_{year}_{i}.nc')
            ds.append(ds_temp)
elif n_seg == 1:
    ds = [ds_in]

#print(ds)

In [130]:
# extract plotting variables & GM

t_stamp = int(ds[0].t_stamp)
depth = ds[0].depth.values
f = ds[0].specf.values

# GM spectrum

gm = np.loadtxt('../project/archive/GM/gm.dat')   # get GM data from separate code
gm_freq = gm[:,0]                                # GM based on N at -904 m
gm_psd = gm[:,1]

In [131]:
# select and average time

# set date range

start_date = dt.datetime(t_stamp,1,1)                                   # input start date in YYYY,MM,DD
end_date = dt.datetime(t_stamp+1,1,1)                                   # input end date in YYYY,MM,DD

dt_list,phi_u,phi_v = [],[],[]
for i in range(n_seg):
    dt_temp = pd.Series(pd.to_datetime(ds[i].spect.values))
    t_seg = len(dt_temp)
    for j in range(t_seg):
        if dt_temp[j] >= start_date and dt_temp[j] <= end_date:
            dt_list.append(ds[i].spect[j].values)
            phi_u.append(ds[i].Sxxu[:,:,j])                 # time segment, [depth, freq]
            phi_v.append(ds[i].Sxxv[:,:,j])
avg_u = sum(phi_u)/len(dt_list)
avg_v = sum(phi_v)/len(dt_list)
            
print("Output time range:",dt_list[0],'to',dt_list[-1])     # print to check desired interval

Output time range: 2018-01-06T08:07:30.000000000 to 2018-12-15T00:37:29.000000000


## Depth-average PSD (annual)

In [132]:
# WKB scale and depth average

WKB_u = avg_u / ds[0].WKB_scale
WKB_v = avg_v / ds[0].WKB_scale

depth_u = WKB_u.mean(dim='depth')
depth_v = WKB_v.mean(dim='depth')

In [133]:
# error bars (95% confidence intervals) for each depth

probability = 0.95                            # calculate confidence intervals
alpha = 1 - probability        
NS = ds[0].t / (ds[0].nps / 2)             # number of estimates, Welch
vp = (4/3)*NS                                 # for tapered windows
cp = chi2.ppf([1 - alpha / 2, alpha / 2], vp) # chi**2 distribution
cint = vp/cp                                  # interval coefficients

u_lower = depth_u * cint[0]             # define upper and lower confidence values
u_upper = depth_u * cint[1]
v_lower = depth_v * cint[0]             # define upper and lower confidence values
v_upper = depth_v * cint[1]
tot_lower = (depth_u + depth_v) * cint[0]             # define upper and lower confidence values
tot_upper = (depth_u + depth_v) * cint[1]

In [136]:
# plot PSD

fig, ax = plt.subplots(1,1,figsize=(12,7))

ax.axvline(1.161e-5, color ='lawngreen',ls='--',lw=0.9,label = "$K_1$") # constituents
ax.axvline(1.732e-5, color ='cyan',ls='--', lw=0.9,label = "$f$")    
ax.axvline(2.236e-5, color ='orange',ls='--',lw=0.9,label = "$M_2$")
ax.axvline(3.968e-5, color ='gold',ls='--',lw=0.9,label = "$fM_2$")
ax.axvline(4.472e-5, color ='pink',ls='--',lw=0.9,label = "$M_4$")
ax.set_ylim(1e-2,5e3)          # set y limits for comparing PSD
ax.set_xlim(1.27e-6, 5.55e-4)  # first non-zero freq bin to the Nyquist freq 

ax.tick_params(axis='both', direction='in', which='both')
ax.set_title(f'Depth-average PSD - {adcp} - {t_stamp}')
ax.set_ylabel('Power spectral density [$(m/s)^2/Hz$]',fontsize=11)
ax.set_xlabel('Frequency [Hz]',fontsize=11)

if adcp == 'Axis75' or adcp == 'Slope':
    ax.axhline(2.3e-2,color='grey',ls=':',lw=0.9,label='Noise floor')
elif adcp == 'Axis55':
    ax.axhline(0.53,color='grey',ls=':',lw=0.9,label='Noise floor')

ax.loglog(gm_freq/(2*np.pi),gm_psd*(np.pi),color='grey',lw=0.9,ls='--', label='GM79 - 1/2 amp.') # 1/2 GM for components
ax.loglog(gm_freq/(2*np.pi),gm_psd*(2*np.pi),color='grey', label='GM79') # GM  

if adcp == 'Slope':
    ax.loglog(f, depth_u, label='Cross-slope',color='blue',lw=0.9,ls='--')
    ax.loglog(f, depth_v, label='Along-slope',color='red',lw=0.9,ls='--')
    ax.loglog(f, (depth_u + depth_v), label='Total',color='black')
elif adcp == 'Axis55' or adcp == 'Axis75':
    ax.loglog(f, depth_u, label='Cross-canyon',color='blue',lw=0.9,ls='--')
    ax.loglog(f, depth_v, label='Along-canyon',color='red',lw=0.9,ls='--')
    ax.loglog(f, (depth_u + depth_v), label='Total',color='black')

ax.fill_between(f, u_lower, u_upper, facecolor='blue', alpha=0.1)
ax.fill_between(f, v_lower, v_upper, facecolor='red', alpha=0.1)
ax.fill_between(f, tot_lower, tot_upper, facecolor='black', alpha=0.2, label = '95% conf. int.')

fig.tight_layout()
plt.legend(loc='upper right',frameon=False,fontsize=9)
plt.show()

plt.savefig(fname=f'./plots/psd_plots/psd_{adcp}_{t_stamp}.pdf',format='pdf')

<IPython.core.display.Javascript object>

## Depth comparison PSD (annual)

In [ ]:
# select depths and WKB scale

dupidx = 0                                     # upper depth
dlowidx = -1                                   # lower depth
dup_stamp = -depth[dupidx]                     # stamps for plotting
dlow_stamp = -depth[dlowidx]
print('Upper depth:',dup_stamp)
print('Lower depth:',dlow_stamp)

u_up = avg_u[dupidx].values / ds[0].WKB_scale[dupidx].values                    # data for plotting, WKB scaled
v_up = avg_v[dupidx].values / ds[0].WKB_scale[dupidx].values
u_low = avg_u[dlowidx].values / ds[0].WKB_scale[dlowidx].values
v_low = avg_v[dlowidx].values / ds[0].WKB_scale[dlowidx].values

In [ ]:
# error bars (95% confidence intervals) for each depth

probability = 0.95                            # calculate confidence intervals
alpha = 1 - probability        
NS = ds[0].t / (ds[0].nps / 2)             # number of estimates, Welch
vp = (4/3)*NS                                 # for tapered windows
cp = chi2.ppf([1 - alpha / 2, alpha / 2], vp) # chi**2 distribution
cint = vp/cp                                  # interval coefficients

u_low_lower = u_low * cint[0]             # define upper and lower confidence values
u_low_upper = u_low * cint[1]
v_low_lower = v_low * cint[0]             # define upper and lower confidence values
v_low_upper = v_low * cint[1]
u_up_lower = u_up * cint[0]               # define upper and lower confidence values
u_up_upper = u_up * cint[1]
v_up_lower = v_up * cint[0]               # define upper and lower confidence values
v_up_upper = v_up * cint[1]

In [ ]:
# plot PSD

fig, ax = plt.subplots(1,1,figsize=(12,7))

ax.axvline(1.161e-5, color ='lawngreen',ls='--',lw=0.8,label = "$K_1$") # constituents
ax.axvline(1.732e-5, color ='cyan',ls='--', lw=0.8,label = "$f$")    
ax.axvline(2.236e-5, color ='orange',ls='--',lw=0.8,label = "$M_2$")
ax.axvline(3.968e-5, color ='gold',ls='--',lw=0.8,label = "$fM_2$")
ax.axvline(4.472e-5, color ='pink',ls='--',lw=0.8,label = "$M_4$")
ax.set_ylim(1e-2,5e3)          # set y limits for comparing PSD
ax.set_xlim(1.27e-6, 5.55e-4)  # first non-zero freq bin to the Nyquist freq 

ax.tick_params(axis='both', direction='in', which='both')
ax.set_title(f'PSD - {adcp} - {t_stamp}')
ax.set_ylabel('Power spectral density [$(m/s)^2/Hz$]')
ax.set_xlabel('Frequency [Hz]')

if adcp == 'Axis75' or adcp == 'Slope':
    ax.axhline(2.3e-2,color='gray',ls=':',lw=0.8,label='Noise floor')
elif adcp == 'Axis55':
    ax.axhline(0.53,color='gray',ls=':',lw=0.8,label='Noise floor')
    
ax.loglog(gm_freq/(2*np.pi),gm_psd*(np.pi),color='gray',lw=0.8,ls='--', label='GM79 - 1/2 amp.') # 1/2 GM for components

if adcp == 'Slope':
    ax.loglog(f, u_up, label=f'{dup_stamp} m - Cross-slope',color='blue',lw=1)
    ax.loglog(f, v_up, label=f'{dup_stamp} m - Along-slope',color='blue',ls='--',lw=1)
    ax.loglog(f, u_low, label=f'{dlow_stamp} m - Cross-slope',color='red',lw=1)
    ax.loglog(f, v_low, label=f'{dlow_stamp} m - Along-slope',color='red',ls='--',lw=1)
elif adcp == 'Axis55' or adcp == 'Axis75':
    ax.loglog(f, u_up, label=f'{dup_stamp} m - Cross-canyon',color='blue',lw=1)
    ax.loglog(f, v_up, label=f'{dup_stamp} m - Along-canyon',color='blue',ls='--',lw=1)
    ax.loglog(f, u_low, label=f'{dlow_stamp} m - Cross-canyon',color='red',lw=1)
    ax.loglog(f, v_low, label=f'{dlow_stamp} m - Along-canyon',color='red',ls='--',lw=1)

ax.fill_between(f, u_up_lower, u_up_upper, facecolor='blue', alpha=0.1)
ax.fill_between(f, v_up_lower, v_up_upper, facecolor='blue', alpha=0.1)
ax.fill_between(f, u_low_lower, u_low_upper, facecolor='red', alpha=0.1)
ax.fill_between(f, v_low_lower, v_low_upper, facecolor='red', alpha=0.1, label='95% conf. int.')

fig.tight_layout()
plt.legend(loc='upper right',frameon=False,fontsize=9)
plt.show()

# plt.savefig(fname=f'./plots/psd_plots/psd_comp_{adcp}_{t_stamp}.pdf',format='pdf')